In [ ]:
import pandas as pd
import json
import re
from typing import List
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_from_disk
from vllm import LLM, SamplingParams

# 1. Fine-tuning 모델 평가

## 1-1. LLM기반 평가

In [ ]:
import pandas as pd
import json
import re
from typing import List
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_from_disk
from vllm import LLM, SamplingParams

INFO 02-12 10:07:46 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
test_dataset = load_from_disk('test_dataset')
tokenizer = AutoTokenizer.from_pretrained("eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594")

In [ ]:
prompt_lst = []
label_lst=[]
questions=[]
contexts=[]

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    question = text.split('<|start_header_id|>user<|end_header_id|>')[1].split('<|eot_id|>')[0].strip()
    context = text.split('검색 결과:\n-----')[1].split('<|eot_id|>')[0].strip()
    prompt_lst.append(input)
    label_lst.append(label)
    questions.append(question)
    contexts.append(context)

In [ ]:
questions[10]

'진코넥정240밀리그램은 언제 복용해야 하나요?'

In [ ]:
label_lst[10]

'\n검색 결과에는 진코넥정240밀리그램의 복용 시기를 찾을 수 없습니다.'

In [ ]:
sampling_params= SamplingParams(
    temperature=0,
    max_tokens=1024
)

In [ ]:
preds = vllm_model.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 197/197 [00:24<00:00,  8.16it/s, est. speed input: 12787.42 toks/s, output: 532.98 toks/s]


In [ ]:
preds = [pred.outputs[0].text for pred in preds]

In [ ]:
client = OpenAI(api_key = "api key")

In [ ]:
def evaluate_rag_with_llm(questions, contexts, predictions, labels):

    results = []
    
    prompt_template = """
당신은 RAG(Retrieval-Augmented Generation) 시스템 평가 전문가입니다. 아래 정보를 바탕으로 생성된 답변의 품질을 철저히 평가해주세요.

질문: {question}

검색된 컨텍스트:
{context}

생성된 답변:
{prediction}

참조 답변(정답):
{label}

다음 4가지 평가 기준으로 1-5점 척도로 점수를 매겨주세요:

1. 응답 정확성 (Answer Correctness) [1-5]:
   * 생성된 답변이 참조 답변과 비교하여 정확하고 완전한 정보를 제공하는지 평가
   * 1점: 완전히 잘못된 정보
   * 2점: 부분적으로 관련된 정보를 담고 있으나 대부분 부정확함
   * 3점: 정확한 정보와 부정확한 정보가 혼재되어 있음
   * 4점: 대부분 정확하지만 일부 정보가 누락되거나 미미한 오류가 있음
   * 5점: 참조 답변과 비교했을 때 완전히 정확하고 포괄적인 정보를 제공함

2. 컨텍스트 관련성 (Context Relevance) [1-5]:
   * 검색된 컨텍스트가 질문에 대답하기 위해 관련성이 높은지 평가
   * 1점: 컨텍스트가 질문과 전혀 관련이 없음
   * 2점: 컨텍스트가 질문과 간접적으로만 관련됨
   * 3점: 컨텍스트 중 일부만 질문과 직접적으로 관련됨
   * 4점: 대부분의 컨텍스트가 질문과 직접적으로 관련됨
   * 5점: 모든 컨텍스트가 질문에 완벽하게 관련되어 있고 불필요한 정보가 없음

3. 컨텍스트 충실성 (Context Faithfulness) [1-5]:
   * 생성된 답변이 주어진 컨텍스트에만 기반하는지, 아니면 없는 정보를 추가했는지 평가
   * 1점: 답변이 컨텍스트에 없는 정보로만 구성됨 (심각한 환각)
   * 2점: 답변이 주로 컨텍스트에 없는 정보로 구성됨
   * 3점: 답변이 컨텍스트 정보와 없는 정보가 혼합되어 있음
   * 4점: 답변이 주로 컨텍스트에 기반하지만 약간의 추가 정보가 있음
   * 5점: 답변이 전적으로 컨텍스트에 있는 정보만을 사용함

4. 컨텍스트 충분성 (Context Recall) [1-5]:
   * 검색된 컨텍스트가 질문에 완전히 답변하기에 충분한 정보를 포함하는지 평가
   * 1점: 컨텍스트가 답변에 필요한 정보를 전혀 포함하지 않음
   * 2점: 컨텍스트가 필요한 정보의 일부만 포함함
   * 3점: 컨텍스트가 필요한 정보의 약 절반을 포함함
   * 4점: 컨텍스트가 필요한 정보의 대부분을 포함하지만 일부 누락됨
   * 5점: 컨텍스트가 질문에 완전히 답변하기 위한 모든 필요한 정보를 포함함

반드시 다음 JSON 형식으로만 응답하세요. 마크다운은 사용하지 않습니다.:
{
  "answer_correctness": 정수로 된 점수(1-5),
  "context_relevance": 정수로 된 점수(1-5),
  "context_faithfulness": 정수로 된 점수(1-5),
  "context_recall": 점수(1-5),
  "analysis": "종합적인 분석 의견"
}

다른 형식의 응답은 하지 마세요. 오직 마크다운이 아닌 JSON만 반환하세요.
"""
    
    for i in tqdm(range(len(questions)), total=len(questions), desc="RAG 평가 진행 중"):
        try:
            
            prompt = prompt_template
            prompt = prompt.replace("{question}", str(questions[i]) if questions[i] is not None else "")
            prompt = prompt.replace("{context}", str(contexts[i]) if contexts[i] is not None else "")
            prompt = prompt.replace("{prediction}", str(predictions[i]) if predictions[i] is not None else "")
            prompt = prompt.replace("{label}", str(labels[i]) if labels[i] is not None else "")

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "당신은 RAG 평가 도구입니다. 반드시 유효한 JSON 형식으로만 응답하세요."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                response_format={"type": "json_object"}
            )
            
            result = json.loads(response.choices[0].message.content)
            
            answer_correctness = result['answer_correctness']
            context_relevance = result['context_relevance']
            context_faithfulness = result['context_faithfulness']
            context_recall = result['context_recall']
            
            total_score = answer_correctness + context_relevance + context_faithfulness + context_recall
            
            row_result = {
                'id': i,
                'question': questions[i],
                'answer_correctness': answer_correctness,
                'context_relevance': context_relevance,
                'context_faithfulness': context_faithfulness,
                'context_recall': context_recall,
                'total_score': total_score,
                'analysis': result['analysis']
            }
            
            results.append(row_result)
            
        except Exception as e:
            print(f"항목 {i} 평가 중 오류 발생: {e}")
            results.append({
                'id': i,
                'question': questions[i],
                'error': str(e)
            })
    
    results_df = pd.DataFrame(results)
    
    if 'total_score' in results_df.columns:
        metrics_summary = {
            '평균 총점': results_df['total_score'].mean(),
            '응답 정확성 평균': results_df['answer_correctness'].mean(),
            '컨텍스트 관련성 평균': results_df['context_relevance'].mean(),
            '컨텍스트 충실성 평균': results_df['context_faithfulness'].mean(),
            '컨텍스트 충분성 평균': results_df['context_recall'].mean()
        }
        print("\n===== 평가 요약 =====")
        for metric, value in metrics_summary.items():
            print(f"{metric}: {value:.2f}")
    
    return results_df, metrics_summary if 'total_score' in results_df.columns else results_df

In [ ]:
results_df, metrics_summary = evaluate_rag_with_llm(questions, contexts, preds, label_lst)

RAG 평가 진행 중: 100%|██████████| 197/197 [09:59<00:00,  3.04s/it]


===== 평가 요약 =====
평균 총점: 18.71
응답 정확성 평균: 4.85
컨텍스트 관련성 평균: 4.71
컨텍스트 충실성 평균: 4.99
컨텍스트 충분성 평균: 4.15


In [ ]:
results_df.head()

,id,question,answer_correctness,context_relevance,context_faithfulness,context_recall,total_score,analysis
0,0,덴플러스포르테캡슐의 성분에 대해 설명해 줄래?,5,5,5,2,17,생성된 답변은 참조 답변과 동일하게 덴플러스포르테캡슐의 성분에 대한 설명을 찾을 수...
1,1,케어스킨로션을 사용하면 생리통이 완화될까요?,5,5,5,5,20,생성된 답변은 참조 답변과 동일하게 검색된 컨텍스트에서 케어스킨로션이 생리통 완화에...
2,2,테라미플러스크림을 어느 약국에서 구할 수 있는지 아는 사람 있어?,5,5,5,5,20,"생성된 답변은 참조 답변과 완전히 일치하며, 질문에 대한 정확하고 포괄적인 정보를 ..."
3,3,클리어드롭점안액의 유통기한은 얼마인가요?,5,5,5,2,17,"생성된 답변은 참조 답변과 일치하며, 클리어드롭점안액의 유통기한에 대한 정보가 없다..."
4,4,모이스뷰점안액의 사용 금지는 어떤 제품으로부터 영향을 받나요?,5,5,5,2,17,"생성된 답변은 참조 답변과 비교했을 때 정확하고 포괄적인 정보를 제공하고 있으며, ..."


In [ ]:
print(metrics_summary)

{'평균 총점': 18.705583756345177, '응답 정확성 평균': 4.852791878172589, '컨텍스트 관련성 평균': 4.7055837563451774, '컨텍스트 충실성 평균': 4.99492385786802, '컨텍스트 충분성 평균': 4.152284263959391}


## 1-2. 인용문서 기반 평가

In [ ]:
def extract_ref_numbers(text: str)  -> List[int]:
    pattern = r'\[\[ref(\d+)\]\]'

    numbers = [int(match.group(1)) for match in re.finditer(pattern, text)]

    return sorted(list(set(numbers)))

In [ ]:
def calculate_f1_score(labels, predictions) : 

    is_nested_list = isinstance(labels, list) and len(labels) > 0 and isinstance(labels[0], list)

    if not is_nested_list : 
        labels = [labels]
        predictions = [predictions]

    if len(labels) != len(predictions) : 
        raise ValueError("라벨과 예측의 길이가 일치하지 않습니다")

    total_true_positives = 0
    total_false_positives = 0
    total_false_negatives = 0

    for sample_labels, sample_preds in zip(labels, predictions) : 
        if not sample_labels : 
            sample_labels = set()
        else : 
            sample_labels = set(sample_labels)

        if not sample_preds : 
            sample_preds = set()
        else : 
            sample_preds = set(sample_preds)

        true_positives = len(sample_labels.intersection(sample_preds))
        false_positives = len(sample_preds - sample_labels)
        false_negatives = len(sample_labels - sample_preds)

        total_true_positives += true_positives
        total_false_positives += false_positives
        total_false_negatives += false_negatives

    if sum(bool(l) for l in labels) == 0 and sum(bool(p) for p in predictions) == 0 : 
        return 1.0

    precision = total_true_positives / (total_true_positives + total_false_positives) if (total_true_positives + total_false_positives) > 0 else 0
    recall = total_true_positives / (total_true_positives + total_false_negatives) if (total_true_positives + total_false_negatives) > 0 else 0

    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1

In [ ]:
label_ref_numbers = []
pred_ref_numbers = []

for label, pred in zip(label_lst, preds) : 
    label_ref_numbers.append(extract_ref_numbers(label))
    pred_ref_numbers.append(extract_ref_numbers(pred))

In [ ]:
label_ref_numbers[100:110]

[[5], [1], [3], [5], [1], [4], [4], [2], [1], [4]]

In [ ]:
pred_ref_numbers[100:110]

[[5], [1], [3], [5], [1], [4], [4], [2], [1], [4]]

In [ ]:
f1 = calculate_f1_score(label_ref_numbers, pred_ref_numbers)

In [ ]:
print(f1)

0.9847328244274809


# 2. Base Model 평가

## 3-1. LLM 기반 평가

In [ ]:
vllm_model = LLM(
    model="NCSOFT/Llama-VARCO-8B-Instruct",
    dtype="bfloat16",
)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 02-12 11:01:24 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 02-12 11:01:24 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

INFO 02-12 11:01:36 [weight_utils.py:281] Time spent downloading weights for NCSOFT/Llama-VARCO-8B-Instruct: 7.800788 seconds


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 02-12 11:01:39 [loader.py:447] Loading weights took 2.40 seconds
INFO 02-12 11:01:39 [gpu_model_runner.py:1186] Model loading took 14.9596 GB and 10.968036 seconds
INFO 02-12 11:01:48 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/cf07419ff9/rank_0_0 for vLLM's torch.compile
INFO 02-12 11:01:48 [backends.py:425] Dynamo bytecode transform time: 8.45 s
INFO 02-12 11:01:51 [backends.py:132] Cache the graph of shape None for later use
INFO 02-12 11:02:15 [backends.py:144] Compiling a graph for general shape takes 26.59 s
INFO 02-12 11:02:28 [monitor.py:33] torch.compile takes 35.04 s in total
INFO 02-12 11:02:29 [kv_cache_utils.py:566] GPU KV cache size: 417,952 tokens
INFO 02-12 11:02:29 [kv_cache_utils.py:569] Maximum concurrency for 8,192 tokens per request: 51.02x
INFO 02-12 11:02:55 [gpu_model_runner.py:1534] Graph capturing finished in 27 secs, took 0.52 GiB
INFO 02-12 11:02:56 [core.py:151] init engine (profile, create kv cache, warmup model) too

In [ ]:
test_dataset = load_from_disk('test_dataset')
tokenizer = AutoTokenizer.from_pretrained("eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

In [ ]:
prompt_lst = [] # LLM의 입력
label_lst = [] # LLM의 답변
questions = [] # 사용자의 질문
contexts = [] # 검색 결과

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    question = text.split('<|start_header_id|>user<|end_header_id|>')[1].split('<|eot_id|>')[0].strip()
    context = text.split('검색 결과:\n-----')[1].split('<|eot_id|>')[0].strip()
    prompt_lst.append(input)
    label_lst.append(label)
    questions.append(question)
    contexts.append(context)

In [ ]:
questions[70]

'건아이점안액 사용 후 증상이 72시간 이상 지속되면 어떻게 해야 하나요?'

In [ ]:
label_lst[70]

'\n건아이점안액 사용 후 증상이 72시간 이상 지속되면 사용을 중지하고 의사와 상의해야 합니다. [[ref1]]'

In [ ]:
# 생성 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
)

In [ ]:
preds = vllm_model.generate(prompt_lst, sampling_params)

Processed prompts: 100%|██████████| 197/197 [00:39<00:00,  4.99it/s, est. speed input: 7821.18 toks/s, output: 1346.81 toks/s]


In [ ]:
preds = [pred.outputs[0].text for pred in preds]

In [ ]:
results_df, metrics_summary = evaluate_rag_with_llm(questions, contexts, preds, label_lst)

RAG 평가 진행 중: 100%|██████████| 197/197 [10:04<00:00,  3.07s/it]


===== 평가 요약 =====
평균 총점: 17.94
응답 정확성 평균: 4.57
컨텍스트 관련성 평균: 4.57
컨텍스트 충실성 평균: 4.78
컨텍스트 충분성 평균: 4.02


In [ ]:
results_df.head()

,id,question,answer_correctness,context_relevance,context_faithfulness,context_recall,total_score,analysis
0,0,덴플러스포르테캡슐의 성분에 대해 설명해 줄래?,5,5,5,2,17,생성된 답변은 덴플러스포르테캡슐의 성분에 대한 정보가 없음을 정확하게 전달하고 있으...
1,1,케어스킨로션을 사용하면 생리통이 완화될까요?,5,4,5,3,17,생성된 답변은 케어스킨로션이 생리통 완화에 효과적이라는 정보가 없음을 명확히 전달하...
2,2,테라미플러스크림을 어느 약국에서 구할 수 있는지 아는 사람 있어?,4,5,5,2,16,"생성된 답변은 테라미플러스크림에 대한 구체적인 정보를 제공하지 못했지만, 질문에 대..."
3,3,클리어드롭점안액의 유통기한은 얼마인가요?,5,5,5,2,17,생성된 답변은 클리어드롭점안액의 유통기한에 대한 정보가 없음을 정확하게 전달하고 있...
4,4,모이스뷰점안액의 사용 금지는 어떤 제품으로부터 영향을 받나요?,2,3,3,2,10,"생성된 답변은 모이스뷰점안액의 사용 금지와 관련된 정보가 포함되어 있으나, 참조 답..."


In [ ]:
print(metrics_summary)

{'평균 총점': 17.944162436548222, '응답 정확성 평균': 4.573604060913706, '컨텍스트 관련성 평균': 4.573604060913706, '컨텍스트 충실성 평균': 4.776649746192893, '컨텍스트 충분성 평균': 4.020304568527918}


## 2-2. 인용문서 기반 평가

In [ ]:
label_ref_numbers = []
pred_ref_numbers = []

for label, pred in zip(label_lst, preds):
    label_ref_numbers.append(extract_ref_numbers(label))
    pred_ref_numbers.append(extract_ref_numbers(pred))

In [ ]:
label_ref_numbers[100:110]

[[5], [1], [3], [5], [1], [4], [4], [2], [1], [4]]

In [ ]:
pred_ref_numbers[100:110]

[[], [], [3], [], [], [], [4], [], [], []]

In [ ]:
f1 = calculate_f1_score(label_ref_numbers, pred_ref_numbers)

In [ ]:
print(f1)

0.08510638297872342
